In [1]:
%load_ext autoreload
%autoreload 2

import csv
import os
from cyvcf2 import VCF

In [2]:
project_folder = "diygenomics-projects"
sub_category = "DATA"
work_bucket = "genomes"

base_file_name = 'GCF_000001405.40'
input_file = f'{base_file_name}.gz'
output_dir = f'{base_file_name}_chunks'

limit = 1000000
dir_num = 0

In [3]:
data_path = '/Volumes/cold_storage/data' # os.getenv('DATA_PATH')
file_path = lambda *args: os.path.join(data_path, project_folder, sub_category, work_bucket, *args)

In [4]:
os.makedirs(file_path(f"{output_dir}/{dir_num}"), exist_ok=True)

In [5]:
vcf = VCF(file_path(input_file))

In [ ]:
last_rsid = None
file_count = 0

while True:
    new_rows = []
    count = 0
    for variant in vcf:
        rsid = variant.ID
        geneinfo = variant.INFO.get('GENEINFO')

        if rsid == last_rsid:
            continue

        # Prepare rows for writing to CSV
        if geneinfo is None:
            new_rows.append([rsid, None])
        else:
            # Split GENEINFO by '|'
            genes = geneinfo.split('|')

            # Generate a row for each gene
            for gene in genes:
                gene = gene.split(':')[0]
                new_rows.append([rsid, gene])

        # If limit reached, break loop
        count += 1
        if count >= limit:
            last_rsid = rsid
            break

    # If no more rows, break the loop
    if not new_rows:
        break

    # Calculate the directory to write to based on the file_count
    if file_count % 10 == 1:
        dir_num = file_count // 10
        os.makedirs(file_path(f"{output_dir}/{dir_num}"), exist_ok=True)

    # Write to CSV
    with open(file_path(f"{output_dir}/{dir_num}", f'{file_count}.csv'), "w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["RSID", "GENEINFO"])  # write headers
        writer.writerows(new_rows)

    print(f"Written to {dir_num}/{file_count}.csv")

    # Prepare for next iteration
    file_count += 1

In [ ]:
total_records = 1_000_000_000  # 1 billion
records_per_minute = 6_000_000  # 6 million

# Calculate total time in minutes
total_time_minutes = total_records / records_per_minute

# Convert total time to hours and minutes
total_time_hours = total_time_minutes // 60  # This will give the whole number of hours
remaining_minutes = total_time_minutes % 60  # This will give the remaining minutes

print(f"Total time to process the records is {total_time_minutes} minutes.")
print(f"This is equivalent to {total_time_hours} hours and {remaining_minutes} minutes.")